## Build Autoencoder Model

- Code in this notebook creates a convolutional neural network autoencoder model for reproducing artwork images 

### Imports

In [ ]:
import pandas as pd
import numpy as np
import os
import cv2
import pickle
import json
import numpy as np
from scipy import spatial

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Input
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model, load_model
from keras.callbacks import EarlyStopping

import matplotlib.pyplot as plt
%matplotlib inline

### Define Autoencoder Parameters

In [ ]:
epochs = 20
batch_size = 20
image_dimension = 200

In [ ]:
# Input layer
input_img = Input(shape=(image_dimension, image_dimension, 3))

# Encoding layers
x = Conv2D(512, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same', name='encoded_layer')(x)

# Decoding layers
x = Conv2D(16, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

# Instanstiate Model
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mse', metrics=['acc'])

In [ ]:
# View parameter summary
autoencoder.summary()

### Transform Images for Model

In [ ]:
# Directories for images
base_directory = './autoencoder/'
training_directory = os.path.join(base_directory, 'train/')
validation_directory = os.path.join(base_directory, 'validate/')
test_directory = os.path.join(base_directory, 'test/')

In [ ]:
# Sample files used to train model drawn from all images
sample_size = 10000
allimage_directory = './autoencoder/all_data/'
total_image_count = len([name for name in os.listdir(allimage_directory)])
all_image_names = os.listdir(allimage_directory)
sample_indices = np.random.randint(0, total_image_count, size=sample_size)
training_sample_indices = sample_indices[:int(sample_size*.8)]
validation_sample_indices = sample_indices[int(sample_size*.8):]

In [ ]:
# Copy images into training and validation folders
from shutil import copy2
for training_sample_index in training_sample_indices:
    copy2(allimage_directory + all_image_names[training_sample_index], training_directory + all_image_names[training_sample_index])
for validation_sample_index in validation_sample_indices:
    copy2(allimage_directory + all_image_names[validation_sample_index], validation_directory + all_image_names[validation_sample_index])

In [ ]:
# Create generators to process images
# Rescale by 1./255
datagen = ImageDataGenerator(rescale=1./255)

training_generator = datagen.flow_from_directory(
        training_directory,
        target_size=(image_dimension, image_dimension),
        batch_size=batch_size,
        # No classes since unsupervised
        class_mode='input')

validation_generator = datagen.flow_from_directory(
        validation_directory,
        target_size=(image_dimension, image_dimension),
        batch_size=batch_size,
        # No classes since unsupervised
        class_mode='input')

### Train Model

In [ ]:
# Early stopping if epochs don't change loss
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='auto'

In [ ]:
# Number of samples for calculating number of steps
training_samples = training_generator.samples
validation_samples = validation_generator.samples

# Number of steps
training_steps = training_samples / batch_size
validation_steps = validation_samples / batch_size

In [ ]:
# Train model
history = autoencoder.fit_generator(
      training_generator,
      steps_per_epoch=training_steps,
      epochs=epochs,
      validation_data=validation_generator,
      validation_steps=validation_steps,
      callbacks=[early_stopping])

### Save Model and Narrow Encoded Layer

In [ ]:
# Save model
autoencoder.save('./autoencoder/models/autoencoder.h5')

In [ ]:
# Save feature extractor
feature_extractor = \
    Model(inputs=autoencoder.input, outputs=autoencoder.get_layer('encoded_layer').output)
feature_extractor.save('./autoencoder/models/feature_extractor.h5')

In [ ]:
# Plot accuracy and loss for training and validation sets
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()